In [ ]:
import qiskit
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi

import numpy as np

In [ ]:
def ghz_circuit_noise(num_qubits, error_h=None, error_cx=None):
    """Generate a noisy GHZ preparation circuit"""
    from qiskit.extensions import HGate, CXGate

    # Noisy H-gate
    if error_h is None:
        h_gate = HGate()
    else:
        h_gate = qiskit.QuantumCircuit(1, name='h_noise')
        h_gate.append(HGate(), [0])
        h_gate.append(error_h, [0])
        h_gate = h_gate.to_instruction()

    # Noisy CX-gate
    if error_cx is None:
        cx_gate = CXGate()
    else:
        cx_gate = qiskit.QuantumCircuit(2, name='cx_noise')
        cx_gate.append(CXGate(), [0, 1])
        cx_gate.append(error_cx, [0, 1])
        cx_gate = cx_gate.to_instruction()
        
    # Make noisy circuit
    circ = qiskit.QuantumCircuit(num_qubits)
    circ.append(h_gate, [0])
    for j in range(1, num_qubits):
        circ.append(cx_gate, [0, j])
    return circ
    

In [ ]:
"""
ERROR MODEL
Bit flip errors after the H gate and on each qubit after a CX gate
all iid Bernoulli(p) errors
"""

# Error parameters
I = qi.Operator.from_label('I') #identity gate
X = qi.Operator.from_label('X')
p_error = 0.1

# H error term
chan1 = (1 - p_error) * qi.SuperOp(I) + p_error * qi.SuperOp(X)

# CX error term
chan2 = chan1.tensor(chan1)

In [ ]:
#transpiling noisy circuit gives original gates and noise terms
circ_noise = ghz_circuit_noise(3, error_h=chan1, error_cx=chan2)
circ_noise.draw()